# Run analysis on the data

In [2]:
%matplotlib qt

In [3]:
%run -i ~/Documents/tomology/tomography/utils.py

/home/xf28id2/conda_envs/khalifah_20210725/lib/python3.9/site-packages/pims/image_reader.py:26: RuntimeWarning: PIMS image_reader.py could not find scikit-image. Falling back to matplotlib's imread(), which uses floats instead of integers. This may break your scripts. 
(To ignore this warning, include the line "warnings.simplefilter("ignore", RuntimeWarning)" in your script.)
  warnings.warn(RuntimeWarning(ski_preferred))


In [4]:
import pyFAI
import pathlib

from ipywidgets import interact_manual

## Create the calculator

In [5]:
C = Calculator()

## Load the Run Data

In [6]:
from databroker import catalog

db = catalog["xpdd"]

In [7]:
UID = 'fef1f2f7-301c-41c6-a775-d89e1fe0af23'
run = db[UID]
C.metadata = dict(run.metadata["start"])

In [8]:
data = run.primary.to_dask()
C.frames_arr = data["dexela_image"]
C.frames_arr

,Array,Chunk
Bytes,157.42 GiB,91.12 MiB
Shape,"(1769, 1, 3888, 3072)","(1, 1, 3888, 3072)"
Count,5307 Tasks,1769 Chunks
Type,float64,numpy.ndarray


In [53]:
# show the total intensity distribution
total_i = data["dexela_stats1_total"].values.reshape(C.metadata["shape"])
fig, ax = plt.subplots()
xr.DataArray(total_i).plot.imshow(ax=ax)
ax.invert_yaxis()
plt.show()

## Load the calibration data

In [12]:
PONI_FILE = "/nsls2/data/xpd/legacy/processed/from_mehmet/CeO2_0.25x0.25_beam.poni"
C.ai = pyFAI.load(PONI_FILE) if PONI_FILE else pyFAI.azimuthalIntegrator.AzimuthalIntegrator(dist=200, wavelength=0.186, detector="perkin")

## Choose the Max image

In [16]:
interact_manual(
    show_npy_array, 
    template="/nsls2/data/xpd/legacy/processed/xpdUser/tiff_base/CG-0046_fef1f2f7-301c-41c6-a775-d89e1fe0af23/max/CG-0046_{}_max.npy",
    index=(1427, 1511)
)

interactive(children=(Text(value='/nsls2/data/xpd/legacy/processed/xpdUser/tiff_base/CG-0046_fef1f2f7-301c-41c…

<function __main__.show_npy_array(template: str, index: int, **kwargs)>

In [17]:
C.light = np.load("/nsls2/data/xpd/legacy/processed/xpdUser/tiff_base/CG-0046_fef1f2f7-301c-41c6-a775-d89e1fe0af23/max/CG-0046_1482_max.npy")
C.dark = np.load("/nsls2/data/xpd/legacy/processed/xpdUser/tiff_base/CG-0046_fef1f2f7-301c-41c6-a775-d89e1fe0af23/min/CG-0046_1482_min.npy")

## Subtract the dark from the light

In [18]:
C.light -= C.dark

## Locate the peaks and create windows

In [8]:
# tp.locate?

In [19]:
C.calc_peaks_from_light_frame(25, separation=10, threshold=30, topn=80)

In [23]:
C.calc_windows_from_peaks(num=80, width=40)
C.show_windows(size=10, vmax=100)
plt.show()

In [37]:
C.show_windows(size=10, vmax=100)
plt.show()

## Run throught the images to calculate the intensity in the windows

In [29]:
C.calc_intensity_in_windows()

In [31]:
C.metadata["shape"] = (61, 29)
C.metadata["extents"] = [[28.0, 43.0], [61.0, 68.0]]

In [32]:
C.reshape_intensity()
C.calc_coords()
C.assign_q_values()

In [34]:
intensity = C.intensity.copy()

In [35]:
import matplotlib as mpl

In [36]:
C.intensity = intensity
facet = C.show_intensity(col_wrap=20, sharey=False, norm=mpl.colors.TwoSlopeNorm(400))
axes = facet.axes.flatten()
for i in range(C.intensity.shape[0]):
    d = 2 * np.pi / C.windows["Q"]
    axes[i].set_title("d = {:.3f} nm".format(d.iloc[i]))
    axes[i].invert_yaxis()
plt.show()

## Save the result

In [33]:
BASE = "/nsls2/data/xpd/legacy/processed/xpdUser/tiff_base/"
FILENAME = pathlib.Path(BASE).joinpath("{}_{}.nc".format(C.metadata["sample_name"], C.metadata["uid"]))
C.to_dataset().to_netcdf(FILENAME)

## Load the result again

In [10]:
BASE = "/nsls2/data/xpd/legacy/processed/xpdUser/tiff_base/"
FILENAME = pathlib.Path(BASE).joinpath("{}_{}.nc".format(C.metadata["sample_name"], C.metadata["uid"]))
ds = xr.load_dataset(FILENAME)

In [15]:
ds["intensity"].plot(col="grain", col_wrap=4)